<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оконные функции
**Оконные функции (window functions)** — функции, которые работают по группе строк, но при этом не сводят результат к одной строке (в отличие от `GROUP BY`).

## Основные оконные функции

| Функция        | Назначение                       |
| -------------- | -------------------------------- |
| `ROW_NUMBER()` | Порядковый номер строки в группе |
| `RANK()`       | Ранг с пропусками                |
| `DENSE_RANK()` | Ранг без пропусков               |
| `LAG()`        | Предыдущее значение              |
| `LEAD()`       | Следующее значение               |
| `SUM()`        | Накопительный итог               |
| `AVG()`        | Среднее по окну                  |

## ОКОННАЯ ФУНКЦИЯ SUM(...) OVER vs. GROUP BY

Представим таблицу `orders_log`:

| user\_uuid | order\_time | total\_uah |
| ---------- | ----------- | ---------- |
| u1         | 2024-01-10  | 300        |
| u1         | 2024-01-15  | 400        |
| u2         | 2024-01-12  | 100        |
| u2         | 2024-01-22  | 500        |


Результат `SUM(...) OVER`  

| user\_uuid | order\_time | total\_uah | cumulative\_total |
| ---------- | ----------- | ---------- | ----------------- |
| u1         | 2024-01-10  | 300        | 300               |
| u1         | 2024-01-15  | 400        | 700               |
| u2         | 2024-01-12  | 100        | 100               |
| u2         | 2024-01-22  | 500        | 600               |

Результат `GROUP BY`

| user\_uuid | total\_by\_user |
| ---------- | --------------- |
| u1         | 700             |
| u2         | 600             |

## Принципы написания оконных функций

### Синтаксис

```sql
SELECT column,
       функция(...) OVER (
        ...
       )
FROM таблица;
```


```sql
ФУНКЦИЯ(...) OVER (
    [PARTITION BY разделение_по_группам]
    [ORDER BY порядок_внутри_группы]
    [ROWS BETWEEN ... AND ...]
)
```

| Элемент            | Обязательно? | Назначение                                                                |
| ------------------ | ------------ | ------------------------------------------------------------------------- |
| `ФУНКЦИЯ(...)`     | ✅ Да         | Оконная функция (`SUM`, `ROW_NUMBER`, `LAG`, и т.д.)                      |
| `OVER (...)`       | ✅ Да         | Обязательный блок, который делает функцию оконной                         |
| `PARTITION BY ...` | ❌ Нет        | Разделение на подгруппы (например, по `user_id`)                          |
| `ORDER BY ...`     | ❌ Нет        | Определяет порядок строк в пределах каждой группы                         |
| `ROWS BETWEEN ...` | ❌ Нет        | Уточняет окно: какие строки из отсортированной группы участвуют в расчёте |

### Компоненты оконной функции

#### `ФУНКЦИЯ(...)`

Это агрегатная или оконная функция, применяемая **к окну строк** (не ко всей таблице). Например:

* `SUM()`, `AVG()`, `COUNT()`
* `ROW_NUMBER()`, `RANK()`, `LAG()`, `LEAD()`

#### `OVER (...)` — "окно", по которому будет работать функция

Ключевое отличие от обычной агрегации: **результат остаётся в каждой строке**, а не сворачивается.

##### `PARTITION BY`

* Делит данные на **группы** (разбиения).
* Каждая группа обрабатывается отдельно.

📌 Аналог `GROUP BY`, но результат **не сворачивается**, а остаётся построчно.

**Пример:**

```sql
SUM(total_uah) OVER (PARTITION BY user_uuid)
```

Считает сумму для каждого пользователя по всем его строкам.

##### `ORDER BY`

* Упорядочивает строки **внутри каждой группы (partition)**.
* Обязательно для функций, которым важен **порядок**: `ROW_NUMBER()`, `RANK()`, `LAG()`, `LEAD()`, `CUMULATIVE SUM`.

📌 Если **не указать `ORDER BY`**, то:

* Для `LAG`, `LEAD`, `ROW_NUMBER` и других **порядок будет неопределён** — результат будет некорректным.
* Для `SUM(...) OVER (PARTITION BY ...)` без `ORDER BY` — будет сумма **по всей группе**, но без накопления.

##### `ROWS BETWEEN ... AND ...` — диапазон строк

Определяет, **какой диапазон строк** внутри "окна" включить для расчёта.

**Общая форма:**

```sql
ROWS BETWEEN <начало_окна> AND <конец_окна>
```
---

**Возможные значения**

| Значение              | Что означает                             |
| --------------------- | ---------------------------------------- |
| `UNBOUNDED PRECEDING` | Самое начало окна (первая строка группы) |
| `n PRECEDING`         | `n` строк до текущей                     |
| `CURRENT ROW`         | Текущая строка                           |
| `n FOLLOWING`         | `n` строк после текущей                  |
| `UNBOUNDED FOLLOWING` | До самого конца окна                     |

---

**Примеры комбинаций**

| Синтаксис                                                  | Что делает                                                      |
| ---------------------------------------------------------- | --------------------------------------------------------------- |
| `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`         | Накопительная сумма от начала до текущей строки                 |
| `ROWS BETWEEN 3 PRECEDING AND CURRENT ROW`                 | Скользящее окно из 4 строк: текущая и 3 перед ней               |
| `ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING`                 | Соседи вокруг строки (3 строки: до, текущая, после)             |
| `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`         | От текущей строки до конца группы                               |
| `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` | Вся группа целиком (эквивалентна `PARTITION BY`, но с ORDER BY) |
| `ROWS BETWEEN CURRENT ROW AND CURRENT ROW`                 | Только текущая строка (по умолчанию, если ничего не указано)    |

📌 Если указано `ORDER BY`, но не указано `ROWS`, по умолчанию:

  ```sql
  ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ```

##### Пример

```sql
SELECT user_uuid,
       order_time,
       total_uah,
       SUM(total_uah) OVER (
           PARTITION BY user_uuid
           ORDER BY order_time
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS cumulative_total
FROM orders_log;
```

| Элемент                  | Пояснение                                       |
| ------------------------ | ----------------------------------------------- |
| `SUM(total_uah)`         | Функция: хотим посчитать накопительную сумму    |
| `PARTITION BY user_uuid` | Для каждого пользователя отдельно               |
| `ORDER BY order_time`    | Считаем **в хронологическом порядке** заказов   |
| `ROWS BETWEEN ...`       | Накопление: с самого первого до текущего заказа |
| `AS cumulative_total`    | Название нового столбца с результатом           |


#### Типичные ошибки

| Ошибка                                                         | Что пойдет не так                                                        |
| -------------------------------------------------------------- | ------------------------------------------------------------------------ |
| **Не указали `ORDER BY` для `ROW_NUMBER()` или `LAG()`**       | Порядок будет случайным                                                  |
| **Не указали `PARTITION BY`, а он был нужен**                  | Все пользователи будут в одной группе                                    |
| **Не указали `ROWS` в агрегатных функциях (`SUM()`, `AVG()`)** | В PostgreSQL будет всё от начала до текущей строки, но лучше писать явно |

#### Как писать оконные функции правильно

1. **Определи**, по каким группам работает логика (`PARTITION BY`)
2. **Определи порядок** внутри группы (`ORDER BY`)
3. Для агрегатов — **уточни окно строк** (`ROWS BETWEEN`)
4. Убедись, что **функция применима к текущей задаче**

## Оконные функции

### Тестовая таблица `orders_log`

| user\_id | order\_date | amount |
| -------- | ----------- | ------ |
| u1       | 2024-01-01  | 100    |
| u1       | 2024-01-10  | 150    |
| u1       | 2024-01-20  | 200    |
| u2       | 2024-01-05  | 300    |
| u2       | 2024-02-01  | 100    |

### 1. `SUM(...) OVER (PARTITION BY ...)` — сумма по пользователю

```sql
SELECT user_uuid, order_time, total_uah,
      SUM(total_uah) OVER (
        PARTITION BY user_uuid
      )
FROM orders_log;
```

**Что происходит:**

* Каждой строке присваивается сумма всех `total_uah` по этому `user_uuid`.
* Порядок не важен — здесь просто агрегат без сортировки.

**Результат:**

| user\_uuid | order\_time | total_uah | sum |
| -------- | ----------- | ------ | --- |
| u1       | 2024-01-01  | 100    | 450 |
| u1       | 2024-01-10  | 150    | 450 |
| u1       | 2024-01-20  | 200    | 450 |
| u2       | 2024-01-05  | 300    | 400 |
| u2       | 2024-02-01  | 100    | 400 |


### 2. 📈 `SUM(...) OVER (PARTITION BY ... ORDER BY ...)` — кумулятивная сумма

```sql
SELECT user_uuid, order_time, total_uah,
      SUM(total_uah) OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      )
FROM orders_log;
```

**Что происходит:**

* Сначала данные разбиваются по `user_uuid`
* Затем в каждой группе сортируются по `order_time`
* Считается нарастающая сумма

**Результат:**

| user\_uuid | order\_time | total_uah | sum |
| -------- | ----------- | ------ | --- |
| u1       | 2024-01-01  | 100    | 100 |
| u1       | 2024-01-10  | 150    | 250 |
| u1       | 2024-01-20  | 200    | 450 |
| u2       | 2024-01-05  | 300    | 300 |
| u2       | 2024-02-01  | 100    | 400 |


### 3. `LAG()` и `LEAD()` — предыдущее и следующее значение

```sql
SELECT user_uuid, order_time, total_uah,
       LAG(total_uah) OVER (PARTITION BY user_uuid ORDER BY order_time),
       LEAD(total_uah) OVER (PARTITION BY user_uuid ORDER BY order_time)
FROM orders_log;
```

**Объяснение:**

* `LAG(...)` — вернёт значение предыдущей строки внутри группы
* `LEAD(...)` — вернёт значение следующей строки внутри группы

**Результат:**

| user\_uuid | order\_time | total\_uah | lag  | lead |
| -------- | ----------- | ------ | ---- | ---- |
| u1       | 2024-01-01  | 100    | null | 150  |
| u1       | 2024-01-10  | 150    | 100  | 200  |
| u1       | 2024-01-20  | 200    | 150  | null |
| u2       | 2024-01-05  | 300    | null | 100  |
| u2       | 2024-02-01  | 100    | 300  | null |


### 4. `ROW_NUMBER()` — порядковый номер в группе

```sql
SELECT user_uuid, order_time, total_uah,
      ROW_NUMBER() OVER (
        PARTITION BY user_uuid ORDER BY order_time
      )
FROM orders_log;
```

**Что делает:**

* Пронумерует строки **внутри каждой группы по `user_uuid`**
* Порядок задаётся по дате

**Результат:**

| user\_uuid | order\_time | total\_uah | row\_number |
| -------- | ----------- | ------ | ----------- |
| u1       | 2024-01-01  | 100    | 1           |
| u1       | 2024-01-10  | 150    | 2           |
| u1       | 2024-01-20  | 200    | 3           |
| u2       | 2024-01-05  | 300    | 1           |
| u2       | 2024-02-01  | 100    | 2           |


### 5. `RANK()` и `DENSE_RANK()` — ранжирование

```sql
SELECT user_uuid, total_uah,
      RANK() OVER (
          ORDER BY total_uah DESC
      ) AS rank,
      DENSE_RANK() OVER (
          ORDER BY total_uah DESC
      ) AS dense_rank
FROM orders_log;
```

**Разница:**

* `RANK()` — пропускает ранги (1, 1, 3)
* `DENSE_RANK()` — не пропускает (1, 1, 2)

**Результат:**

| user\_uuid | total\_uah | rank | dense\_rank |
| ---------- | ---------- | ---- | ----------- |
| u2         | 300        | 1    | 1           |
| u1         | 200        | 2    | 2           |
| u1         | 200        | 2    | 2           |
| u1         | 150        | 4    | 3           |
| u2         | 100        | 5    | 4           |
| u1         | 100        | 5    | 4           |

### 6. `FIRST_VALUE()` и `LAST_VALUE()`

```sql
SELECT user_uuid, order_time, total_uah,
      FIRST_VALUE(total_uah) OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      ),
      LAST_VALUE(total_uah) OVER (
         PARTITION BY user_uuid
         ORDER BY order_time
         ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
      )
FROM orders_log;
```

**Что делает:**

* `FIRST_VALUE(...)` — первое значение по заданному `ORDER BY`
* `LAST_VALUE(...)` — последнее, но нужно указать границы окна

**Результат:**

| user\_uuid | order\_time | total\_uah | first\_value | last\_value |
| -------- | ----------- | ------ | ------------ | ----------- |
| u1       | 2024-01-01  | 100    | 100          | 200         |
| u1       | 2024-01-10  | 150    | 100          | 200         |
| u1       | 2024-01-20  | 200    | 100          | 200         |
| u2       | 2024-01-05  | 300    | 300          | 100         |
| u2       | 2024-02-01  | 100    | 300          | 100         |

## Задачи

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [3]:
DB_USER = "da_test_user"
DB_PASS = "c89M2psBdXGtBXpY3BPrTRZ2bXl6yq2i"
DB_HOST = "dpg-d1hvnobe5dus739ipbcg-a.frankfurt-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "da_test"

In [4]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

**Task 1. Кумулятивная выручка по каждому пользователю**  
Как накапливалась сумма заказов у каждого пользователя по времени?

**Task 2. Интервал между покупками**  
Сколько дней прошло с предыдущего заказа?

*Цель:* Выявить пользователей с длинными перерывами — возможно, они уходят.

**Task 3. Порядковый номер заказа**  
Какая по счёту это покупка для пользователя?

**Task 4. Последняя сумма покупки (для сравнения)**  
Какая была последняя покупка пользователя? Как она отличается от текущей?

**Task 5. Самый крупный заказ — на каком месте он произошёл?**  
Самый большой заказ был первым, вторым или позже?

*Цель:* Понять, есть ли эффект "затухания" или наоборот — рост с доверием.

**Task 6. Первый заказ каждого пользователя**  
Какова сумма первого заказа?

**Task 7. Признак повторной покупки**  
Сделал ли пользователь хотя бы 2 заказа?

#  Когортный анализ  
Когда мы анализируем поведение пользователей во времени, средние значения могут вводить в заблуждение:
*  одни пользователи только что пришли, другие — давно ушли.
*  нам нужен способ смотреть, как изменяется поведение пользователей от момента их первого действия.

## Что такое когорта?

**Когорта** — это группа пользователей, которые объединились по дате первого действия:

| Пользователь | first\_seen (дата установки) | Когорта    |
| ------------ | ---------------------------- | ---------- |
| A            | 2024‑03‑01                   | 2024‑03‑01 |
| B            | 2024‑03‑01                   | 2024‑03‑01 |
| C            | 2024‑03‑02                   | 2024‑03‑02 |

---

## Для чего нужен когортный анализ?

Когортный анализ отвечает на вопросы:

*  Когда и где мы теряем пользователей?
*  Как меняется поведение пользователей после установки?
*  Когда пользователи начинают приносить выручку?

---

## Виды когорт

| Тип когорты            | Пример                                |
| ---------------------- | ------------------------------------- |
| По дате установки      | Все, кто установил приложение в марте |
| По дате авторизации    | Все, кто вошёл в аккаунт в апреле     |
| По дате первой покупки | Все, кто впервые купил в мае          |

## Retention
**Retention Day N** — сколько пользователей из когорты вернулись на `N`-й день после установки.

### Формула

$$
\text{Retention}_{n} = \frac{\text{Число пользователей, активных на день }n}{\text{Общее число в когорте}}
$$

---

### Структура когортной таблицы

| Когорта / День → | День 0 | День 1 | День 2 | День 3 |
| ---------------- | ------ | ------ | ------ | ------ |
| 2024‑03‑01       | 100    | 40     | 25     | 10     |
| 2024‑03‑02       | 80     | 35     | 15     | 5      |

* Строка = когорта (дата установки)
* Столбец = день активности после установки

---

### Какие таблицы используются?

| Таблица             | Для чего используется               |
| ------------------- | ----------------------------------- |
| `app_sessions`      | Дата установки (когорта)            |
| `product_views_log` | Дата активности (просмотры товаров) |

### Когортный анализ выручки

* Устанавливаем когорту по дате установки (`first_seen`)
* Присоединяем покупки через `devices_users_map` и `orders_log`
* Считаем, сколько дней прошло между установкой и покупкой
* Группируем по `cohort_date` и `days_since_install`, суммируя выручку

In [13]:
query = """
SELECT
  app_sessions.first_seen::date AS cohort_date,
  (orders_log.order_time - app_sessions.first_seen)::int AS days_since_install,
  SUM(orders_log.total_uah) AS revenue
FROM app_sessions
JOIN devices_users_map
  ON app_sessions.device_code = devices_users_map.device_code
JOIN orders_log
  ON devices_users_map.user_uuid = orders_log.user_uuid
WHERE app_sessions.first_seen BETWEEN '2024-03-01' AND '2024-03-31'
GROUP BY cohort_date, days_since_install
ORDER BY cohort_date, days_since_install;
"""

df = pd.read_sql(text(query), engine)
df

,cohort_date,days_since_install,revenue
0,2024-03-01,-57,2537.92
1,2024-03-01,-48,502.71
2,2024-03-01,-35,2687.07
3,2024-03-01,-27,1279.29
4,2024-03-01,-20,2493.18
...,...,...,...
423,2024-03-31,59,479.02
424,2024-03-31,64,1653.87
425,2024-03-31,75,1804.96
426,2024-03-31,80,1435.34


In [14]:
# Преобразуем в когортную матрицу
cohort_matrix = df.pivot_table(
    index="cohort_date",
    columns="days_since_install",
    values="revenue",
    fill_value=0
)

# Нормализуем по дню 0 — считаем Retention %
retention_matrix = cohort_matrix.divide(cohort_matrix[0], axis=0).round(3)

# Сбросим индекс для визуализации
retention_long = retention_matrix.reset_index().melt(
    id_vars="cohort_date",
    var_name="days_since_install",
    value_name="retention_rate"
)

fig = px.imshow(
    retention_matrix,
    labels=dict(x="День с установки", y="Дата установки", color="Retention"),
    x=retention_matrix.columns,
    y=retention_matrix.index,
    color_continuous_scale="Blues",
    text_auto=True
)

fig.update_layout(
    title="📊 Retention по когортам (по просмотрам товаров)",
    xaxis_title="День с момента установки",
    yaxis_title="Дата установки (когорта)",
    yaxis=dict(autorange="reversed")
)

fig.show()


#### **Каждая строка (Y)** — когорта пользователей, установивших приложение в один день  
Например, строка `2024‑03‑10` — это все пользователи, установившие приложение 10 марта.  

#### **Каждый столбец (X)** — день после установки

Например, `0` — день установки, `1` — следующий день, `7` — неделя спустя и т.д.

#### **Цвет ячейки** — выручка от пользователей этой когорты в конкретный день

Чем темнее цвет → тем больше выручка.

---

#### Что ты можешь узнать с такого графика:

##### ✅ **Когда когорты приносят деньги**

* Если тёмные ячейки находятся ближе к дню `0` или `1`, значит **пользователи покупают сразу** после установки.
* Если они распределены позже — значит **покупки происходят спустя некоторое время** (это может быть важно для оценки LTV).

##### ✅ **Как отличаются когорты**

* Например, когорта 31 марта явно принесла больше выручки, чем 17 марта → стоит понять, **какой канал привлечения** использовался тогда.

##### ✅ **Повторные покупки**

* Если выручка тянется в дальние дни (день 20, 30, 50…), значит пользователи **возвращаются и совершают новые заказы**.
* Если всё сосредоточено в `day 0–1`, значит продукт — **одноразовый**, и стоит задуматься об удержании.

##### ✅ **Сезонность и изменения в продукте**

* Вдруг с какой-то даты когорты стали резко приносить меньше — это может быть связано с **изменениями в UX**, **багами** или **новым каналом привлечения**.

---

#### Возможные выводы:

1. **Быстрые покупки?** — Да/Нет (по активности в первые дни).
2. **Есть ли "длинные хвосты"?** — Дни спустя 10+ → признак долгосрочной ценности.
3. **Какая когорта самая прибыльная?** — По насыщенности цвета.
4. **Нужно ли работать над удержанием?** — Если выручка резко обрывается → да.